In [1]:
import numpy as np
import pandas as pd
import math 
from pymoo.factory import get_problem

from pymoo.interface import crossover
from pymoo.factory import get_crossover
from pymoo.interface import mutation
from pymoo.factory import get_mutation

pf = get_problem("truss2d").pareto_front()
phi1= pf[:,0].reshape(-1,1)
phi2= pf[:,1].reshape(-1,1)
m =pf.shape[0]
print(phi1)
print(phi2)

[[0.004     ]
 [0.00404076]
 [0.00408152]
 ...
 [0.05125235]
 [0.05131968]
 [0.05138701]]
[[100000.        ]
 [ 98991.22689126]
 [ 98002.60298646]
 ...
 [  8437.34734651]
 [  8435.03206743]
 [  8432.74042712]]


In [2]:
def select_mating_pool(pop, fitness, num_parents):

    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.

    parents = np.empty((num_parents, pop.shape[1]))

    for parent_num in range(num_parents):

        min_fitness_idx = np.where(fitness == np.min(fitness))

        min_fitness_idx = min_fitness_idx[0][0]

        parents[parent_num, :] = pop[min_fitness_idx, :]

        fitness[min_fitness_idx] = 99999

    return parents

In [3]:
def fitness(b1,b2,d):
    fitness_pop = np.empty([pop_size,1])
    fitness_pop[:]=np.nan
    
    for i in range(pop_size):

        d= d.astype(int)
        c  = np.array((phi1**b1[i])*(phi2**b2[i]))
        X  =((c.max()-c.min())/d[i])


        y=np.full((d[i],1100), np.nan)
        for j in range(d[i]-1):
            y[j,:c[np.logical_and(c>=j*X+c.min(),c<(j+1)*X+c.min())].size]=c[np.logical_and(c>=j*X+c.min(),c<(j+1)*X+c.min())]
        y[d[i]-1,:c[np.logical_and(c>=(d[i]-1)*X+c.min(),c<=c.max())].size]=c[np.logical_and(c>=(d[i]-1)*X+c.min(),c<=c.max())]
        
        p=np.copy(y)



        import math
        C_=0
        U_=0

        count1= np.count_nonzero(~np.isnan(p[0]))
        if count1<= math.floor(1100/d[i]) :
            U_=U_+count1
        else :
            C_=C_+1

        for k in range (1,d[i]):
            count= np.count_nonzero(~np.isnan(p[k]))
            count_=np.count_nonzero(~np.isnan(p[k-1]))

            if count<= math.floor(1100/d[i]) :
                U_=U_+count
            elif count_<=math.floor(1100/d[i]) :
                C_=C_+1
            else:
                p[k][count:count+count_]=p[k-1][:count_]
                p[k-1]= np.full((1,1100), np.nan)

        p.sort(axis=1)

        p=pd.DataFrame(p)
        e=p.isnull().all(axis=1)

        for l in range(d[i]):
            if e[l]==True:
                p=p.drop([l])

        p=p.to_numpy()
        h=p.shape[0]


        _cv=np.full((1,h),np.nan)
        for m in range(h):
            count = np.count_nonzero(~np.isnan(p[m]))
            if count> math.floor(1100/d[i]) :
                _cv[0][m]= (100*np.nanstd(p[m]))/np.nanmean(p[m])
            else:
                _cv[0][m] = 0

        #fitness
        if U_!=0:
            fitness_pop[i] = (C_+U_+np.nansum(_cv))+1000
        else:
            fitness_pop[i] = (C_+U_+np.nansum(_cv))

    return fitness_pop

In [4]:
import random
pop_size=100
d=random.randint(1,1100)
b2=random.random()
pop= np.empty([pop_size,3])
for i in range(pop_size):
    pop[i][0]=random.random()
    pop[i][1]=random.random()
    pop[i][2]=random.randint(1,1100)
    
for i in range(pop_size):
    if pop[i][0]>=pop[i][1]:
        pop[i][1]=pop[i][1]/pop[i][0]
        pop[i][0]=1
    else:
        pop[i][0]=pop[i][0]/pop[i][1]
        pop[i][1]=1
print(pop)

[[1.00000000e+00 9.45032531e-01 6.68000000e+02]
 [1.72749698e-01 1.00000000e+00 1.04000000e+03]
 [1.00000000e+00 4.67473813e-01 8.70000000e+01]
 [1.00000000e+00 5.10098025e-01 4.71000000e+02]
 [1.00000000e+00 8.89238119e-01 4.55000000e+02]
 [6.16067084e-01 1.00000000e+00 5.23000000e+02]
 [1.00000000e+00 4.55833176e-01 4.54000000e+02]
 [1.42116623e-01 1.00000000e+00 7.02000000e+02]
 [7.08767706e-01 1.00000000e+00 9.57000000e+02]
 [5.90896745e-01 1.00000000e+00 9.13000000e+02]
 [2.77810247e-01 1.00000000e+00 1.02200000e+03]
 [5.38861224e-01 1.00000000e+00 9.01000000e+02]
 [1.00000000e+00 2.17158638e-01 4.56000000e+02]
 [7.77179923e-01 1.00000000e+00 1.01200000e+03]
 [2.58728049e-01 1.00000000e+00 7.12000000e+02]
 [5.90934046e-01 1.00000000e+00 1.05900000e+03]
 [1.56329884e-01 1.00000000e+00 3.07000000e+02]
 [4.91156851e-01 1.00000000e+00 4.80000000e+01]
 [4.78740567e-01 1.00000000e+00 1.00200000e+03]
 [1.00000000e+00 1.98708541e-01 4.20000000e+02]
 [1.00000000e+00 1.51793119e-01 5.480000

In [5]:
for i in range(100):
    f=fitness(pop[:,0], pop[:,1],pop[:,2])
    a=select_mating_pool(pop,f,50) 
    b=a.copy()
    np.random.shuffle(b)
    
    u=np.delete(a,(1,2),1) #b1
    v=np.delete(b,(1,2),1) #b1
    g=np.delete(a,(0,2),1) #b2
    h=np.delete(b,(0,2),1) #b2
    s=np.delete(a,(0,1),1) #d
    t=np.delete(b,(0,1),1) #d



    off_b1 = crossover(get_crossover("real_sbx", prob=0.9, eta=10, prob_per_variable=1.0), u, v)
    off_b2 = crossover(get_crossover("real_sbx", prob=0.9, eta=10, prob_per_variable=1.0), g, h)
    off_d  =  crossover(get_crossover("int_sbx", prob=0.9, eta=10, prob_per_variable=1.0), s, t, xl=2, xu=1100)

    off_b1m = mutation(get_mutation("real_pm", eta=50, prob=0.05), off_b1)
    off_b2m = mutation(get_mutation("real_pm", eta=50, prob=0.05), off_b2)
    off_dm = mutation(get_mutation("int_pm", eta=50, prob=0.5), off_d, xl=2, xu=1100)

    pop=np.concatenate((off_b1m,off_b2m,off_dm),axis=1)
    
    for j in range(pop_size):
        if pop[j][0]>=pop[j][1]:
            pop[j][1]=pop[j][1]/pop[j][0]
            pop[j][0]=1
        else:
            pop[j][0]=pop[j][0]/pop[j][1]
            pop[j][1]=1
    
    best_index=np.argmin(fitness(pop[:,0],pop[:,1],pop[:,2]))
    
    print("gen:",i,"=",pop)
    print("best solution of gen:",i,"=",pop[best_index])


gen: 0 = [[4.35151655e-01 1.00000000e+00 8.00000000e+00]
 [1.00000000e+00 1.51502736e-01 8.84000000e+02]
 [1.20996866e-01 1.00000000e+00 4.38000000e+02]
 [9.54764500e-01 1.00000000e+00 5.74000000e+02]
 [7.66366440e-02 1.00000000e+00 6.10000000e+02]
 [1.00000000e+00 9.86228029e-01 6.73000000e+02]
 [4.88984093e-01 1.00000000e+00 3.37000000e+02]
 [2.17538964e-01 1.00000000e+00 3.98000000e+02]
 [1.00000000e+00 2.61996403e-01 3.63000000e+02]
 [7.44749142e-02 1.00000000e+00 9.50000000e+01]
 [9.63483049e-02 1.00000000e+00 1.77000000e+02]
 [1.82898840e-01 1.00000000e+00 3.82000000e+02]
 [9.94659142e-01 1.00000000e+00 1.09000000e+02]
 [9.98808354e-01 1.00000000e+00 6.04000000e+02]
 [3.31981846e-01 1.00000000e+00 1.79000000e+02]
 [2.12355885e-01 1.00000000e+00 6.38000000e+02]
 [9.38453114e-01 1.00000000e+00 8.00000000e+00]
 [3.64939119e-01 1.00000000e+00 6.51000000e+02]
 [1.44005411e-01 1.00000000e+00 7.03000000e+02]
 [1.58242187e-01 1.00000000e+00 3.08000000e+02]
 [3.44040777e-01 1.00000000e+00

gen: 2 = [[9.18688301e-01 1.00000000e+00 5.29000000e+02]
 [1.80097111e-01 1.00000000e+00 6.98000000e+02]
 [1.00000000e+00 9.84774496e-01 6.40000000e+02]
 [9.99250848e-01 1.00000000e+00 6.18000000e+02]
 [1.14279224e-01 1.00000000e+00 3.65000000e+02]
 [1.16596680e-01 1.00000000e+00 6.01000000e+02]
 [1.00000000e+00 9.96657769e-01 6.57000000e+02]
 [1.00000000e+00 9.98615129e-01 5.64000000e+02]
 [9.99891087e-01 1.00000000e+00 1.14000000e+02]
 [9.91143689e-01 1.00000000e+00 2.44000000e+02]
 [9.85173259e-01 1.00000000e+00 7.11000000e+02]
 [9.83343979e-01 1.00000000e+00 1.29000000e+02]
 [4.99368782e-01 1.00000000e+00 6.45000000e+02]
 [9.79619184e-01 1.00000000e+00 1.35000000e+02]
 [1.00000000e+00 2.52604900e-01 5.66000000e+02]
 [1.00000000e+00 9.57705037e-01 6.41000000e+02]
 [1.13515899e-01 1.00000000e+00 5.79000000e+02]
 [1.91940835e-01 1.00000000e+00 5.66000000e+02]
 [1.38085862e-01 1.00000000e+00 1.30000000e+01]
 [4.42218450e-01 1.00000000e+00 5.60000000e+02]
 [1.95509732e-01 1.00000000e+00

gen: 98 = [[  0.99289959   1.         576.        ]
 [  0.99289958   1.         575.        ]
 [  0.99289704   1.         589.        ]
 [  0.99745951   1.         595.        ]
 [  0.99289884   1.         606.        ]
 [  0.99289915   1.         559.        ]
 [  0.99289906   1.         620.        ]
 [  0.99289707   1.         560.        ]
 [  0.99289908   1.         563.        ]
 [  0.99289903   1.         561.        ]
 [  1.           0.9808159  634.        ]
 [  0.99289904   1.         562.        ]
 [  0.99289905   1.         552.        ]
 [  0.99289942   1.         621.        ]
 [  0.99289855   1.         546.        ]
 [  0.99289883   1.         570.        ]
 [  0.99289885   1.         561.        ]
 [  0.99289878   1.         577.        ]
 [  0.99289878   1.         576.        ]
 [  0.99289825   1.         592.        ]
 [  0.99289868   1.         554.        ]
 [  0.99289867   1.         666.        ]
 [  0.99289866   1.         573.        ]
 [  1.           0.99838

In [11]:
b1=pop[best_index][0]
b2=pop[best_index][1]
d= pop[best_index][2]
d= d.astype(int)
print("b1=",b1,"b2=",b2,"d=",d)

c  = np.array((phi1**b1)*(phi2**b2))
X  =((c.max()-c.min())/d)


y=np.full((d,1100), np.nan)
for j in range(d-1):
    y[j,:c[np.logical_and(c>=j*X+c.min(),c<(j+1)*X+c.min())].size]=c[np.logical_and(c>=j*X+c.min(),c<(j+1)*X+c.min())]
y[d-1,:c[np.logical_and(c>=(d-1)*X+c.min(),c<=c.max())].size]=c[np.logical_and(c>=(d-1)*X+c.min(),c<=c.max())]

p=np.copy(y)



import math
C_=0
U_=0

count1= np.count_nonzero(~np.isnan(p[0]))
if count1<= (math.floor(1100/d)+3) :
    U_=U_+count1
else :
    C_=C_+1

for k in range (1,d):
    count= np.count_nonzero(~np.isnan(p[k]))
    count_=np.count_nonzero(~np.isnan(p[k-1]))

    if count<= (math.floor(1100/d)+3) :
        U_=U_+count
    elif count_<=(math.floor(1100/d)+3) :
        C_=C_+1
    else:
        p[k][count:count+count_]=p[k-1][:count_]
        p[k-1]= np.full((1,1100), np.nan)
        
Significance= (1100-U_)/11
print("Significance percentage =",Significance,"%")


b1= 0.9929210123530718 b2= 1.0 d= 573
Significance percentage = 83.45454545454545 %
